# Analize weather data from my handmade weatherstation

Подгружаем основные библиотеки для работы с данным

Настраиваем отображение графиков и отключаем вывод предупреждений

In [1]:
# import libraries
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import time
import datetime

from pd_names import names as names
from myutils import temp_eff as te
from myutils import dew_point as dp

import warnings
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'svg'
%matplotlib inline

In [2]:
colors = ['black', 'silver', 'firebrick', 'sandybrown', 'gold', 'olivedrab',
          'chartreuse', 'green', 'cyan', 'darkblue', 'coral', 'orange',
          'yellow', 'red', 'greenyellow', 'steelblue']

In [3]:
# load data and preprocessing
# index_col = 'date', parse_date=True при открытии указываем, 
# что индексом является дата записи и необходимо распарсить дату
%time ds = pd.read_csv('../data/weather_2018.csv', names=names[:-1]+['day'], sep='\t').iloc[1:,:]
ds = ds.dropna()

CPU times: user 4.32 s, sys: 668 ms, total: 4.99 s
Wall time: 5.07 s


Преобразуем столбец date к типу datetime64

Устанавливаем date  в качестве index

Данные хранятся в raw формате:
 - все показатели целочисленные:
 - температура уличная  (два десятичных знака, градусы Цельсия)
 - влажность  (два десятичных знака, %)
 - давление  (один десятичный знак, мм.рт.ст.)
 - освещенность  (целое число 0 - 65534, Люксы)
 - температура в помещении  (два десятичных знака, градусы Цельсия)
 - напряжение питания модуля  (3 десятичных знака, миллиВольты)
 - радиационный фон  (целое число, мкР/час)
 - датчик широкого спектра газов  (целое число, 0 - 1024 бит)
 - датчик инфракрасного излучения  (целое число, 0 - 1024 бит)
 - датчик скорости ветра  (два десятичных знака, м/с)

In [4]:
ds.date = ds.date.astype('datetime64')
ds['month'] = ds.date.dt.month
ds['hour'] = ds.date.dt.hour
ds['wday'] = ds.date.dt.weekday
ds.set_index('date', inplace=True)
ds.tout = ds.tout*0.01
ds.hout = ds.hout*0.01
ds.pout = ds.pout*0.1
ds.tin = ds.tin*0.01
ds.wind_cur = ds.wind_cur*0.01
# ограничим естественный диапазон значений
ds = ds[ds.uR > 0]
ds = ds[ds.wind_cur >= 0]
ds= ds[(ds.pout < 820) & (ds.pout > 650)]

Преобразуем дни месяца и часы к целочисленному виду

Декодируем значения дней недели в привычные имена

In [5]:
ds['month'] = ds.month.map({
    1.0:'january', 2.0:'february', 3.0:'march', 4.0:'april',
    5.0:'may', 6.0:'june', 7.0:'july', 8.0:'august', 
    9.0:'september', 10.0:'october', 11.0:'november', 12.0:'december'
})
ds['hour'] = ds.hour.map(int)
ds['wday'] = ds.wday.map({0.0:'monday', 1.0:'tuesday', 
                            2.0:'wednesday', 3.0:'thursday',
                            4.0:'friday', 5.0:'saturday', 6.0:'sunday'})

Добавляем столбцы "точка росы", "эффективная температура" и "дефицит точки росы"

In [6]:
ds['dew_point'] = dp(ds.tout, ds.hout)
ds['temp_eff'] = te(ds.tout, ds.hout, ds.wind_cur, lux=0)
ds['dif_dew_point'] = ds.tout - ds['dew_point']

In [7]:
# глянем на данные
ds.head()

,tout,hout,pout,lout,tin,vcc,uR,mq2,infrared,wind_cur,day,month,hour,wday,dew_point,temp_eff,dif_dew_point
date,,,,,,,,,,,,,,,,,
2018-01-01 00:00:29,1.7,99.9,747.4,0,23.25,4957.0,23.0,183,-1.0,0.0,2018-01-01,january,0,monday,1.825155,-4.949332,-0.125155
2018-01-01 00:00:50,1.7,99.9,747.4,0,23.25,4913.0,16.0,183,-1.0,0.0,2018-01-01,january,0,monday,1.825155,-4.949332,-0.125155
2018-01-01 00:01:11,1.8,99.9,747.4,0,23.25,4957.0,28.0,184,-1.0,0.0,2018-01-01,january,0,monday,1.925260,-4.866573,-0.125260
2018-01-01 00:01:32,1.7,99.9,747.4,0,23.25,4957.0,12.0,184,-1.0,0.0,2018-01-01,january,0,monday,1.825155,-4.949332,-0.125155
2018-01-01 00:01:53,1.7,99.9,747.4,0,23.25,4935.0,13.0,184,-1.0,0.0,2018-01-01,january,0,monday,1.825155,-4.949332,-0.125155


In [8]:
# проверим есть ли пропуски
# если нет идем дальше
ds.isna().count()

tout             1423992
hout             1423992
pout             1423992
lout             1423992
tin              1423992
vcc              1423992
uR               1423992
mq2              1423992
infrared         1423992
wind_cur         1423992
day              1423992
month            1423992
hour             1423992
wday             1423992
dew_point        1423992
temp_eff         1423992
dif_dew_point    1423992
dtype: int64

Задача состоит в предсказании будет ли мороз ночью или нет
Пока мы не знаем какие показатели могут влиять на заморозки
Точно можно сказать какие влиять не будут (tin, vcc, uR, mq2, infrared) и все категориальные
Отберем нужные нам

In [9]:
data = ds[['tout', 'hout', 'pout', 'lout', 'wind_cur', 'dew_point', 'temp_eff', 'dif_dew_point', 'hour']]

Опишем деление дня на временные промежутки:
- ночь с 0 до 9
- утро с 9 до 12
- день с 12 до 18
- вечер с 18 до 00
Решаем задачу предсказания вероятности заморозка ночью следующего дня

Предсказания необходимо знать до 18 вечера текущего дня

Исходя из вышеописанного, необходимо:
- выделить ночной интервал каждого дня, для каждого дня пометить "был ли мороз
- выделить утренний и дневной интервалы с усреднением в течении часа
- сформировать данные следующим образом (день средние показатели целевая переменная)

# Ресемплируем данные, аггрегируя в течении часа

In [10]:
data = data.resample('1H', how='mean')
data = data.dropna()

In [11]:
# посмотрим
data.head()

,tout,hout,pout,lout,wind_cur,dew_point,temp_eff,dif_dew_point,hour
date,,,,,,,,,
2018-01-01 00:00:00,1.605488,99.9,747.450000,0.0,0.0,1.730544,-5.027704,-0.125056,0.0
2018-01-01 01:00:00,1.384884,99.9,747.523837,0.0,0.0,1.509710,-5.210853,-0.124826,1.0
2018-01-01 02:00:00,1.364912,99.9,747.705848,0.0,0.0,1.489717,-5.227452,-0.124805,2.0
2018-01-01 03:00:00,1.367857,99.9,747.905952,0.0,0.0,1.492665,-5.225003,-0.124808,3.0
2018-01-01 04:00:00,1.371345,99.9,747.953801,0.0,0.0,1.496157,-5.222112,-0.124812,4.0


In [12]:
# создадим колонку "day"
data['day'] = data.index.map(lambda x: str(x)[:10])
data.hour = data.hour.map(int)

In [13]:
data.head()

,tout,hout,pout,lout,wind_cur,dew_point,temp_eff,dif_dew_point,hour,day
date,,,,,,,,,,
2018-01-01 00:00:00,1.605488,99.9,747.450000,0.0,0.0,1.730544,-5.027704,-0.125056,0,2018-01-01
2018-01-01 01:00:00,1.384884,99.9,747.523837,0.0,0.0,1.509710,-5.210853,-0.124826,1,2018-01-01
2018-01-01 02:00:00,1.364912,99.9,747.705848,0.0,0.0,1.489717,-5.227452,-0.124805,2,2018-01-01
2018-01-01 03:00:00,1.367857,99.9,747.905952,0.0,0.0,1.492665,-5.225003,-0.124808,3,2018-01-01
2018-01-01 04:00:00,1.371345,99.9,747.953801,0.0,0.0,1.496157,-5.222112,-0.124812,4,2018-01-01


In [14]:
# делаем выборку утро+день и ночь
data_day = data[(data.hour >= 9) & (data.hour < 18)]
data_night = data[(data.hour >= 0) & (data.hour < 9)]

In [15]:
# проверяем правильно ли отобраны данные
print(set(data_day.hour))
print(set(data_night.hour))

{9, 10, 11, 12, 13, 14, 15, 16, 17}
{0, 1, 2, 3, 4, 5, 6, 7, 8}


In [16]:
# аггрегируем данные, отбирая минимальную температуру за ночь
# создаем целевую переменную "freez" бфл мороз или нет
# также убираем запись о первом дне года, так как мы не знаем что было днем ранее
days_below_zero = data_night.groupby('day', as_index=False).agg({'tout':'min'})
days_below_zero['freez'] = days_below_zero.tout < 0
days_below_zero.freez = days_below_zero.freez.map(int)
days_below_zero = days_below_zero.iloc[1:, :]

In [17]:
days_below_zero.head()

,day,tout,freez
1,2018-01-02,1.746541,0
2,2018-01-03,1.576190,0
3,2018-01-04,1.119767,0
4,2018-01-05,-1.567251,1
5,2018-01-06,-1.303509,1


In [18]:
# работаем с data_day
data_day.head()

,tout,hout,pout,lout,wind_cur,dew_point,temp_eff,dif_dew_point,hour,day
date,,,,,,,,,,
2018-01-01 09:00:00,1.014024,99.9,747.710976,330.878049,0.0,1.138463,-5.519954,-0.124439,9,2018-01-01
2018-01-01 10:00:00,1.291279,99.9,747.986628,1038.162791,0.0,1.416007,-5.288861,-0.124728,10,2018-01-01
2018-01-01 11:00:00,1.759064,99.9,748.078947,1538.666667,0.0,1.884281,-4.900517,-0.125217,11,2018-01-01
2018-01-01 12:00:00,1.997771,99.9,748.262048,1330.132530,0.0,2.123238,-4.703246,-0.125467,12,2018-01-01
2018-01-01 13:00:00,1.937733,99.9,747.975000,1464.976744,0.0,2.063137,-4.752994,-0.125404,13,2018-01-01


In [19]:
# соберем в таблицу все метеоданные за каждый час для каждого дня
days_features = data_day.pivot_table(
                    index='day', 
                    columns='hour', 
                    values='tout', 
                    aggfunc='mean'
                    ).rename(columns={i:'tout_'+str(i)+'h' for i in range(9, 18)})

In [20]:
# в цикле смержим с остальными показателями
for col in data_day[['hout', 'pout', 'lout', 'wind_cur', 'dew_point', 'temp_eff', 'dif_dew_point']]:
    temp_days_features = data_day.pivot_table(
                    index='day', 
                    columns='hour', 
                    values=col, 
                    aggfunc='mean'
                    ).rename(columns={i:str(col)+'_'+str(i)+'h' for i in range(9, 18)})
    days_features = days_features.merge(temp_days_features, on='day', how='outer')

In [21]:
# на выходе имеем все показатели для данного дня
days_features.head()

hour,tout_9h,tout_10h,tout_11h,tout_12h,tout_13h,tout_14h,tout_15h,tout_16h,tout_17h,hout_9h,...,temp_eff_17h,dif_dew_point_9h,dif_dew_point_10h,dif_dew_point_11h,dif_dew_point_12h,dif_dew_point_13h,dif_dew_point_14h,dif_dew_point_15h,dif_dew_point_16h,dif_dew_point_17h
day,,,,,,,,,,,,,,,,,,,,,
2018-01-01,1.014024,1.291279,1.759064,1.997771,1.937733,1.652047,1.675756,0.863743,-0.034731,99.9,...,-6.401891,-0.124439,-0.124728,-0.125217,-0.125467,-0.125404,-0.125105,-0.125131,-0.124282,-0.123348
2018-01-02,2.538554,2.811243,3.012426,3.288415,3.183721,3.020468,2.804678,2.757143,2.700000,99.9,...,-4.126725,-0.126034,-0.126320,-0.126532,-0.126822,-0.126712,-0.126540,-0.126313,-0.126263,-0.126203
2018-01-03,1.555814,1.876550,2.316168,2.697605,3.137278,3.250588,2.975439,2.744444,2.544767,99.9,...,-4.253697,-0.125005,-0.125340,-0.125801,-0.126201,-0.126663,-0.126782,-0.126493,-0.126250,-0.126040
2018-01-04,1.212281,1.318563,1.379882,1.438462,1.310180,1.398830,1.087209,1.010526,1.091813,99.9,...,-5.455001,-0.124646,-0.124757,-0.124821,-0.124882,-0.124748,-0.112096,-0.124515,-0.124435,-0.124520
2018-01-05,-1.952635,-1.953554,-2.102105,-2.027879,-1.915569,-1.911696,-1.931395,-1.997024,-1.947953,99.9,...,-8.038138,-0.121364,-0.121363,-0.121210,-0.121287,-0.121403,-0.121407,-0.121386,-0.121319,-0.121369


In [22]:
# теперь необходимо удалить последний день (для него мы не знаем целевую переменную)
days_features = days_features.iloc[:-1, :]

In [23]:
# создаем колонку "freez" и "tout_min"
days_features.reset_index()
data_train = days_features.merge(days_below_zero, on=days_below_zero.index, how='outer').drop('key_0', axis=1)

In [24]:
data_train = data_train.rename(columns={'tout':'target_tout', 'freez':'target_freez'})

In [25]:
# проверяем, что все так, как мы задумывали
data_train.head()

,tout_9h,tout_10h,tout_11h,tout_12h,tout_13h,tout_14h,tout_15h,tout_16h,tout_17h,hout_9h,...,dif_dew_point_11h,dif_dew_point_12h,dif_dew_point_13h,dif_dew_point_14h,dif_dew_point_15h,dif_dew_point_16h,dif_dew_point_17h,day,target_tout,target_freez
0,1.014024,1.291279,1.759064,1.997771,1.937733,1.652047,1.675756,0.863743,-0.034731,99.9,...,-0.125217,-0.125467,-0.125404,-0.125105,-0.125131,-0.124282,-0.123348,2018-01-02,1.746541,0
1,2.538554,2.811243,3.012426,3.288415,3.183721,3.020468,2.804678,2.757143,2.700000,99.9,...,-0.126532,-0.126822,-0.126712,-0.126540,-0.126313,-0.126263,-0.126203,2018-01-03,1.576190,0
2,1.555814,1.876550,2.316168,2.697605,3.137278,3.250588,2.975439,2.744444,2.544767,99.9,...,-0.125801,-0.126201,-0.126663,-0.126782,-0.126493,-0.126250,-0.126040,2018-01-04,1.119767,0
3,1.212281,1.318563,1.379882,1.438462,1.310180,1.398830,1.087209,1.010526,1.091813,99.9,...,-0.124821,-0.124882,-0.124748,-0.112096,-0.124515,-0.124435,-0.124520,2018-01-05,-1.567251,1
4,-1.952635,-1.953554,-2.102105,-2.027879,-1.915569,-1.911696,-1.931395,-1.997024,-1.947953,99.9,...,-0.121210,-0.121287,-0.121403,-0.121407,-0.121386,-0.121319,-0.121369,2018-01-06,-1.303509,1


# Cохраняем датасет

In [26]:
data_train = data_train.dropna()

In [27]:
data_train.to_csv('../data/train.csv', index=False)